# Ferrofluid-Monolayer - Part III 

## Susceptibility with fluctuation formulas

In this part we want to calculate estimators for the initial susceptibility, i.e. the susceptibility at zero external magnetic field. One could carry out several simulations with different external magnetic field strenghts and get the initial susceptibility by fitting a line to the results. We want to go a more elegant way by using fluctuation formulas known from statistical mechanics.
In three dimensions the initial susceptibility $\chi_{init}$ can be calculated with zero field simulations through

\begin{equation}
    \chi_{init} = \frac{V \cdot \mu_0}{3 \cdot k_B T} \left( \langle \boldsymbol{M}^2 \rangle - \langle \boldsymbol{M} \rangle^2 \right) = \frac{\mu_0}{3 \cdot k_B T \cdot V} \left( \langle \boldsymbol{\mu}_{tot}^2 \rangle - \langle \boldsymbol{\mu}_{tot} \rangle^2 \right)
\end{equation}

where $\boldsymbol{M}$ is the magnetization vector. In direction $i$ it reads

\begin{equation}
    M_i = \frac{1}{V} \Bigg\langle \sum_{j=1}^N \mu_j^i \Bigg\rangle
\end{equation}

where $\mu_j^i$ is the $j$ th dipole moment in direction $i$.

For a quasi two dimensional System due to symmetry we have at least two different susceptibilities

\begin{equation}
    \chi_{init,||} = \frac{V \cdot \mu_0}{2 \cdot k_B T} \left( \langle \boldsymbol{M}^2_{||} \rangle - \langle \boldsymbol{M}_{||} \rangle^2 \right) = \frac{\mu_0}{2 \cdot k_B T \cdot V} \left( \langle \boldsymbol{\mu}^2_{tot,||} \rangle - \langle \boldsymbol{\mu}_{tot,||} \rangle^2 \right)
\end{equation}

and 

\begin{equation}
    \chi_{init,\perp} = \frac{V \cdot \mu_0}{k_B T} \left( \langle \boldsymbol{M}^2_{\perp} \rangle - \langle \boldsymbol{M}_{\perp} \rangle^2 \right) = \frac{\mu_0}{k_B T \cdot V} \left( \langle \boldsymbol{\mu}^2_{tot,\perp} \rangle - \langle \boldsymbol{\mu}_{tot,\perp} \rangle^2 \right)
\end{equation}

where $\boldsymbol{\mu}_{tot,||}$ and $\boldsymbol{\mu}_{tot,\perp}$ are the dipole moments of the whole system parallel and perpendicular to the monolayer plane.  
We now want to calculate and compare the initial susceptibility $\chi_{init}$ with the results of Ref. [1] for the monodisperse case. As the system itself is homogen we have the two different susceptibilities $\chi_{init,||}$ and $\chi_{init,\perp}$.

First we import all necessary packages

In [1]:
from __future__ import print_function

import espressomd

from espressomd.magnetostatics import DipolarP3M
from espressomd.magnetostatic_extensions import DLC

from espressomd.cluster_analysis import ClusterStructure
from espressomd.pair_criteria import DistanceCriterion


import numpy as np

Now we setting up all necessary simulation parameters

In [2]:
lj_sigma=1
lj_epsilon=1
lj_cut = 2**(1./6.) * lj_sigma

# magnetic field constant
mu_0 = 4. * np.pi

# Particles
N = 1000

# density
rho = 0.4
# Area fraction of the mono-layer
phi = rho * np.pi * lj_sigma**2. / 4.

# Dipolar interaction parameter lambda = mu_0 m^2 /(4 pi sigma^3 kT)
dip_lambda = 1.

# Temperature
kT =1.0

# Friction coefficient
gamma=1.0

# Time step
dt =0.01

# Verlet list skin
skin =0.4

# System setup
box_size = (N * np.pi * (lj_sigma/2.)**2 /phi)**0.5

# dipole moment
dipm = np.sqrt(dip_lambda*4*np.pi*lj_sigma**3*kT / mu_0)

and the system (for more details see Part I)

In [3]:
system=espressomd.System(box_l=(box_size,box_size,box_size)) 
system.time_step=dt
system.cell_system.skin=skin
system.thermostat.set_langevin(kT=kT,gamma=gamma)
system.set_random_state_PRNG()

# Lennard Jones interaction
system.non_bonded_inter[0,0].lennard_jones.set_params(epsilon=lj_epsilon,sigma=lj_sigma,cutoff=lj_cut, shift="auto")

# Random dipole moments
dip_phi=np.random.random((N,1)) *2. * np.pi
dip_cos_theta = 2*np.random.random((N,1)) -1
dip_sin_theta = np.sin(np.arccos(dip_cos_theta))
dip = np.hstack((
   dip_sin_theta *np.sin(dip_phi),
   dip_sin_theta *np.cos(dip_phi),
   dip_cos_theta))

# Random positions in the monolayer
pos = box_size* np.hstack((np.random.random((N,2)), np.zeros((N,1))))

# Add particles
system.part.add(pos=pos,rotation=N*[(1,1,1)],dip=dip,fix=N*[(0,0,1)])

# Remove overlap between particles by means of the steepest descent method
system.integrator.set_steepest_descent(
    f_max=0,gamma=0.1,max_displacement=0.05)

while system.analysis.energy()["total"] > 5*kT*N:
    system.integrator.run(20)

# Switch to velocity Verlet integrator
system.integrator.set_vv()

# Setup dipolar P3M and dipolar layer correction
system.actors.add(DipolarP3M(accuracy=5E-4,prefactor=dip_lambda*lj_sigma**3*kT))
system.actors.add(DLC(maxPWerror=1E-4, gap_size=box_size-lj_sigma))

Now we equilibrate for a while

In [7]:
print("Equilibration")
for i in range(20):
  system.integrator.run(5000)
  print("Dipolar energy",system.analysis.energy()["dipolar"])

Dipolar P3M tune parameters: Accuracy goal = 5.00000e-04 prefactor = 1.00000e+00
System: box_l = 5.00000e+01 # charged part = 1000 Sum[q_i^2] = 1.00000e+03
Dmesh cao Dr_cut_iL   Dalpha_L     Derr         Drs_err    Dks_err    time [ms]
8    3   2.29664e-01 7.91354e+00 4.94976e-04 3.535e-04 3.464e-04 10      
8    2   2.43117e-01 7.17458e+00 4.97684e-04 3.535e-04 3.503e-04 10      
8    4   2.22938e-01 8.31450e+00 4.98869e-04 3.535e-04 3.520e-04 11      
8    1   2.81555e-01 5.41789e+00 4.95805e-04 3.536e-04 3.476e-04 13      
16   3   1.80322e-01 1.15365e+01 4.97737e-04 3.536e-04 3.503e-04 7       
16   2   1.99162e-01 9.94221e+00 4.98121e-04 3.535e-04 3.509e-04 8       
16   4   1.72248e-01 1.23288e+01 4.96857e-04 3.536e-04 3.491e-04 9       
16   1   2.29664e-01 7.91354e+00 7.75162e-04 3.535e-04 6.898e-04 accuracy not achieved
32   3   1.33128e-01 1.75927e+01 4.91338e-04 3.535e-04 3.412e-04 15      
32   2   1.58486e-01 1.38719e+01 4.94693e-04 3.535e-04 3.460e-04 17      
32   4   1.

As we need the magnetization of our system, we import <tt>MagneticDipoleMoment</tt> from <tt>observables</tt> which returns us the total dipole moment of the system which is the magnetization times the volume of the system.

In [19]:
from espressomd.observables import MagneticDipoleMoment
dipm_tot_calc = MagneticDipoleMoment(ids=system.part[:].id)

Now we set the desired number of loops for the sampling

In [10]:
# Sampling
loops = 1000

and sample the first and second moment of the magnetization or total dipole moment, by averaging over all total dipole moments occuring during the simulation

In [22]:
print('Sampling ...')
# calculate initial total dipole moment
dipm_tot_temp = dipm_tot_calc.calculate()

# initiate variables for sum of first and second moment of total dipole moment
dipm_tot_sum = dipm_tot_temp
dipm_tot_2_sum = np.square(dipm_tot_temp)

# sample dipole moment
for i in range(loops):
    print('i = {}'.format(i))
    system.integrator.run(100)
    dipm_tot_temp = dipm_tot.calculate()
    
    # add current dipole moment
    dipm_tot_sum = np.sum((dipm_tot_sum, dipm_tot_temp), axis=0)
    dipm_tot_2_sum = np.sum((dipm_tot_2_sum, np.square(dipm_tot_temp)), axis=0)

# calculate average first and second moment of total dipole moment
dipm_tot = dipm_tot_sum / loops
dipm_tot_2 = dipm_tot_2_sum / loops

Sampling ...
i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
i = 30
i = 31
i = 32
i = 33
i = 34
i = 35
i = 36
i = 37
i = 38
i = 39
i = 40
i = 41
i = 42
i = 43
i = 44
i = 45
i = 46
i = 47
i = 48
i = 49
i = 50
i = 51
i = 52
i = 53
i = 54
i = 55
i = 56
i = 57
i = 58
i = 59
i = 60
i = 61
i = 62
i = 63
i = 64
i = 65
i = 66
i = 67
i = 68
i = 69
i = 70
i = 71
i = 72
i = 73
i = 74
i = 75
i = 76
i = 77
i = 78
i = 79
i = 80
i = 81
i = 82
i = 83
i = 84
i = 85
i = 86
i = 87
i = 88
i = 89
i = 90
i = 91
i = 92
i = 93
i = 94
i = 95
i = 96
i = 97
i = 98
i = 99
i = 100
i = 101
i = 102
i = 103
i = 104
i = 105
i = 106
i = 107
i = 108
i = 109
i = 110
i = 111
i = 112
i = 113
i = 114
i = 115
i = 116
i = 117
i = 118
i = 119
i = 120
i = 121
i = 122
i = 123
i = 124
i = 125
i = 126
i = 127
i = 128
i = 129
i = 130
i = 131
i = 132
i = 133
i = 134
i = 135
i = 136
i

Now we can calculate estimators of the susceptibility parallel and perpendicular to the monolayer plane

In [26]:
# susceptibility parallel to monolayer plane
chi_p =  mu_0 /(box_size**2. * 2.) * ( (dipm_tot_2[0] * dipm**2. - (dipm_tot[0] * dipm)**2.) + (dipm_tot_2[1] * dipm**2. - (dipm_tot[1] * dipm)**2.) )

In [27]:
# susceptibility perpendicular to monolayer plane
chi_perp = mu_0 /(box_size**2.) * (dipm_tot_2[2] * dipm**2. - (dipm_tot[2] * dipm)**2.)

and print the results

In [35]:
print('chi_parallel\t\t= {}'.format(chi_p))
print('chi_perpendicular\t= {}'.format(chi_perp))

chi_p		= 2.5030936126
chi_perp	= 0.821985480724


Copared with table 1 in Ref. [1] they should be very similar

[1] Tamás Kristóf and István Szalai. “Magnetic properties in monolayers of a model
polydisperse ferrofluid”. In: Phys. Rev. E 72 (4 Oct. 2005), p. 041105. doi: 10.1103/
PhysRevE.72.041105. url: https://link.aps.org/doi/10.1103/PhysRevE.72.
041105.